lazily-evaluated property pattern in python

functools.update_wrapper(wrapper, wrapped) 相当于把wrapped的属性都付给wrapper

In [1]:
import functools

class lazy_property:
    def __init__(self, function):
        self.function = function
        functools.update_wrapper(self, function)

    def __get__(self, obj, owner):
        if obj is None:
            return self
        val = self.function(obj)
        obj.__dict__[self.function.__name__] = val
        return val

In [2]:
def lazy_property2(fn):
    """
    A lazy property decorator.

    The function decorated is called the first time to retrieve the result and
    then that calculated result is used the next time you access the value.
    """
    attr = "_lazy__" + fn.__name__

    @property
    def _lazy_property(self):
        if not hasattr(self, attr):
            setattr(self, attr, fn(self))
        return getattr(self, attr)

    return _lazy_property

In [4]:
class Person:
    def __init__(self, name, occupation):
        self.name = name
        self.occupation = occupation
        self.call_count2 = 0

    @lazy_property
    def relatives(self):
        # Get all relatives, let's assume that it costs much time.
        relatives = "Many relatives."
        return relatives

    @lazy_property2
    def parents(self):
        self.call_count2 += 1
        return "Father and mother"

In [5]:
Jhon = Person('Jhon', 'Coder')

In [6]:
Jhon.name

'Jhon'

In [8]:
Jhon.occupation

'Coder'

In [9]:
sorted(Jhon.__dict__.items())

[('call_count2', 0), ('name', 'Jhon'), ('occupation', 'Coder')]

In [10]:
Jhon.relatives

'Many relatives.'

In [11]:
sorted(Jhon.__dict__.items())

[('call_count2', 0),
 ('name', 'Jhon'),
 ('occupation', 'Coder'),
 ('relatives', 'Many relatives.')]

In [12]:
Jhon.parents

'Father and mother'

In [13]:
sorted(Jhon.__dict__.items())

[('_lazy__parents', 'Father and mother'),
 ('call_count2', 1),
 ('name', 'Jhon'),
 ('occupation', 'Coder'),
 ('relatives', 'Many relatives.')]

In [14]:
Jhon.parents

'Father and mother'

In [15]:
Jhon.call_count2

1